In [1]:
from doubt import Boot
from doubt.datasets import (Airfoil, Blog, Concrete, CPU, 
                            FacebookComments, FishBioconcentration,
                            FishToxicity, ForestFire, NewTaipeiHousing,
                            PowerPlant, Protein, Servo,
                            SpaceShuttle)
from mapie.regression import MapieRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import (LinearRegression, PoissonRegressor, 
                                  GammaRegressor, HuberRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from collections import defaultdict

## Setting up

In [2]:
def evaluate_nasa(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    # Calculate training residuals
    model.fit(X_tr, y_tr)
    tr_preds = model.predict(X_tr)
    te_preds = model.predict(X_te)
    tr_residuals = y_tr - tr_preds
    
    n_train = X_tr.shape[0]
    n_test = X_te.shape[0]

    # Initialise random number generator
    rng = np.random.default_rng(4242)

    # Compute the model variances
    bootstrap_preds = np.empty((n_boots, n_test))
    for boot_idx in range(n_boots):
        train_idxs = rng.choice(range(n_train), size=n_train, replace=True)
        X_btr = X_tr[train_idxs, :]
        y_btr = y_tr[train_idxs]

        model.fit(X_btr, y_btr)
        
        bootstrap_pred = model.predict(X_te)
        bootstrap_preds[boot_idx] = bootstrap_pred

    # Centre the bootstrapped predictions across the bootstrap dimension
    bootstrap_preds = np.mean(bootstrap_preds, axis=0) - bootstrap_preds

    # Add up the bootstrap predictions and the hybrid train/val residuals
    C = np.array([m + o for m in bootstrap_preds for o in tr_residuals])

    # Calculate the intervals
    intervals = np.expand_dims(te_preds, -1) + np.transpose(np.quantile(C, q=[uncertainty/2, 1-uncertainty/2], axis=0))
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [3]:
def evaluate_doubt(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    n_boots = int(np.sqrt(len(X_tr)))
    
    bmodel = Boot(model, random_seed=4242)
    bmodel.fit(X_tr, y_tr, n_boots=n_boots)
    preds, intervals = bmodel.predict(X_te, uncertainty=uncertainty, n_boots=n_boots)
    
    coverage = np.mean((y_te > intervals[:, 0]) & (y_te < intervals[:, 1]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [4]:
def evaluate_mapie(model, X_tr, X_te, y_tr, y_te, uncertainty=0.05):
    bmodel = MapieRegressor(model)
    bmodel.fit(X_tr, y_tr)
    preds, intervals = bmodel.predict(X_te, alpha=uncertainty)
    
    coverage = np.mean((y_te > intervals[:, 0, 0]) & (y_te < intervals[:, 1, 0]))
    mean_width = np.mean(intervals[:, 1] - intervals[:, 0])
    return coverage, mean_width

In [26]:
datasets = []
scaler = StandardScaler()

# Add Doubt datasets
dataset_classes = [
    Airfoil,
    Concrete,
    FishToxicity,
    ForestFire,
    NewTaipeiHousing,
    PowerPlant,
    Protein,
    Servo,
]

for dataset_class in dataset_classes:
    dataset = dataset_class()
    dataset._data = dataset._data.sample(n=min(len(dataset), 1000), random_state=4242)
    X_tr, X_te, y_tr, y_te = dataset.split(test_size=0.1, random_seed=4242)
    X_tr = scaler.fit_transform(X_tr)
    X_te = scaler.transform(X_te)
    datasets.append((dataset_class.__name__, X_tr, X_te, y_tr, y_te))
    
len(datasets)

8

In [27]:
for name, X_tr, X_te, y_tr, y_te in datasets:
    print(f'{name}: {len(X_tr) + len(X_te):,} samples, {X_tr.shape[-1]:,} features')

Airfoil: 1,000 samples, 5 features
Concrete: 1,000 samples, 8 features
FishToxicity: 908 samples, 6 features
ForestFire: 517 samples, 12 features
NewTaipeiHousing: 414 samples, 6 features
PowerPlant: 1,000 samples, 4 features
Protein: 1,000 samples, 9 features
Servo: 167 samples, 4 features


## Linear Regression

In [28]:
model = LinearRegression()

In [31]:
nasa_coverages = list()
nasa_mean_widths = list()
doubt_coverages = list()
doubt_mean_widths = list()

data_dict = defaultdict(list)
for dataset in tqdm(datasets):
    for uncertainty in tqdm([0.01, 0.05, 0.1], leave=False):
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        mapie_coverage, mapie_mean_width = evaluate_mapie(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage_error'].append(100 * (1 - uncertainty - nasa_coverage))
        data_dict['doubt_coverage_error'].append(100 * (1 - uncertainty - doubt_coverage))
        data_dict['mapie_coverage_error'].append(100 * (1 - uncertainty - mapie_coverage))
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
        data_dict['mapie_mean_width'].append(mapie_mean_width)
    
linreg_df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
linreg_df

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

nasa_coverage_error  doubt_coverage_error  \
dataset          uncertainty                                              
Airfoil          0.01                   -0.038462             -1.000000   
                 0.05                    1.730769             -0.192308   
                 0.10                   -1.346154             -1.346154   
Concrete         0.01                   -1.000000             -1.000000   
                 0.05                   -1.153846             -2.115385   
                 0.10                    2.500000              2.500000   
FishToxicity     0.01                    0.075269              0.075269   
                 0.05                   -1.774194             -2.849462   
                 0.10                   -1.397849             -3.548387   
ForestFire       0.01                   -1.000000              0.923077   
                 0.05                   -3.076923             -3.076923   
                 0.10                   -2.307692             -2.307692   
NewTaipeiHousing 0.01                   -1.000000             -1.000000   
                 0.05                   -2.560976             -0.121951   
                 0.10                   -2.682927             -2.682927   
PowerPlant       0.01                   -1.000000             -1.000000   
                 0.05                   -1.153846             -1.153846   
                 0.10                    0.576923             -0.384615   
Protein          0.01                    1.884615             -1.000000   
                 0.05                    3.653846              4.615385   
                 0.10                    2.500000              3.461538   
Servo            0.01                   -1.000000              6.692308   
                 0.05                    2.692308              2.692308   
                 0.10                   20.769231             13.076923   

                              mapie_coverage_error  nasa_mean_width  \
dataset          uncertainty                                          
Airfoil          0.01                    -0.038462        27.458273   
                 0.05                    -0.192308        18.739913   
                 0.10                    -1.346154        15.592097   
Concrete         0.01                    -1.000000        53.752837   
                 0.05                    -0.192308        42.637112   
                 0.10                     0.576923        35.260715   
FishToxicity     0.01                    -1.000000         5.890626   
                 0.05                    -2.849462         4.040719   
                 0.10                    -3.548387         3.057105   
ForestFire       0.01                     0.923077       308.484358   
                 0.05                     0.769231       106.478481   
                 0.10                     5.384615        70.272444   
NewTaipeiHousing 0.01                    -1.000000        51.299282   
                 0.05                    -0.121951        34.991851   
                 0.10                    -2.682927        23.800120   
PowerPlant       0.01                    -1.000000        22.155930   
                 0.05                    -1.153846        16.736078   
                 0.10                    -1.346154        13.973642   
Protein          0.01                    -1.000000        24.783161   
                 0.05                     4.615385        18.894316   
                 0.10                     3.461538        16.137062   
Servo            0.01                     6.692308         6.325876   
                 0.05                    18.076923         3.529766   
                 0.10                    13.076923         3.177570   

                              doubt_mean_width  mapie_mean_width  
dataset          uncertainty                                      
Airfoil          0.01                30.484579         27.840175  
                 0.05          

In [57]:
error_cols = [col for col in linreg_df.columns if 'error' in col]
(linreg_df[error_cols].abs()
                      .describe()
                      .loc[['mean', 'std']]
                      .T
                      .sort_values(by='mean'))

,mean,std
doubt_coverage_error,2.450686,2.756905
nasa_coverage_error,2.453160,4.011081
mapie_coverage_error,3.002037,4.315210


## Decision Tree

In [34]:
model = DecisionTreeRegressor()

In [35]:
nasa_coverages = list()
nasa_mean_widths = list()
doubt_coverages = list()
doubt_mean_widths = list()

data_dict = defaultdict(list)
for dataset in tqdm(datasets):
    for uncertainty in tqdm([0.01, 0.05, 0.1], leave=False):
        nasa_coverage, nasa_mean_width = evaluate_nasa(model, *dataset[1:], uncertainty=uncertainty)
        doubt_coverage, doubt_mean_width = evaluate_doubt(model, *dataset[1:], uncertainty=uncertainty)
        mapie_coverage, mapie_mean_width = evaluate_mapie(model, *dataset[1:], uncertainty=uncertainty)
        data_dict['dataset'].append(dataset[0])
        data_dict['uncertainty'].append(uncertainty)
        data_dict['nasa_coverage_error'].append(100 * (1 - uncertainty - nasa_coverage))
        data_dict['doubt_coverage_error'].append(100 * (1 - uncertainty - doubt_coverage))
        data_dict['mapie_coverage_error'].append(100 * (1 - uncertainty - mapie_coverage))
        data_dict['nasa_mean_width'].append(nasa_mean_width)
        data_dict['doubt_mean_width'].append(doubt_mean_width)
        data_dict['mapie_mean_width'].append(mapie_mean_width)
    
tree_df = pd.DataFrame(data_dict).set_index(['dataset', 'uncertainty'])
tree_df

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

nasa_coverage_error  doubt_coverage_error  \
dataset          uncertainty                                              
Airfoil          0.01                   10.538462             -0.038462   
                 0.05                   11.346154             -0.192308   
                 0.10                   15.000000             -3.269231   
Concrete         0.01                   10.538462             -1.000000   
                 0.05                    8.461538             -3.076923   
                 0.10                   10.192308             -4.230769   
FishToxicity     0.01                   16.204301              2.225806   
                 0.05                   17.580645              3.602151   
                 0.10                   21.182796             -0.322581   
ForestFire       0.01                   37.461538             -1.000000   
                 0.05                   29.615385              0.769231   
                 0.10                   32.307692              3.461538   
NewTaipeiHousing 0.01                   18.512195              3.878049   
                 0.05                   38.902439              4.756098   
                 0.10                   33.902439              2.195122   
PowerPlant       0.01                   26.884615              0.923077   
                 0.05                   18.076923              2.692308   
                 0.10                   23.653846              3.461538   
Protein          0.01                   34.576923              0.923077   
                 0.05                   31.538462              1.730769   
                 0.10                   26.538462              9.230769   
Servo            0.01                   29.769231             -1.000000   
                 0.05                   25.769231             10.384615   
                 0.10                   13.076923             13.076923   

                              mapie_coverage_error  nasa_mean_width  \
dataset          uncertainty                                          
Airfoil          0.01                    -1.000000         9.723192   
                 0.05                    -5.000000         9.683952   
                 0.10                    -8.076923         7.089163   
Concrete         0.01                    -1.000000        21.970042   
                 0.05                    -5.000000        21.435412   
                 0.10                    -7.115385        15.584835   
FishToxicity     0.01                    -1.000000         3.296202   
                 0.05                    -2.849462         2.869638   
                 0.10                    -3.548387         2.232149   
ForestFire       0.01                    -1.000000        95.710950   
                 0.05                    -3.076923       107.830577   
                 0.10                    -2.307692        69.021904   
NewTaipeiHousing 0.01                    -1.000000        18.785598   
                 0.05                    -2.560976        15.580549   
                 0.10                    -5.121951        15.898780   
PowerPlant       0.01                    -1.000000        14.154135   
                 0.05                    -4.038462        14.160673   
                 0.10                    -8.076923        11.286250   
Protein          0.01                    -1.000000        14.965971   
                 0.05                    -5.000000        14.982635   
                 0.10                    -8.076923        12.717654   
Servo            0.01                    -1.000000         1.399055   
                 0.05                     2.692308         1.399055   
                 0.10                     5.384615         1.460597   

                              doubt_mean_width  mapie_mean_width  
dataset          uncertainty                                      
Airfoil          0.01                21.735653         21.281971  
                 0.05          

In [58]:
error_cols = [col for col in linreg_df.columns if 'error' in col]
(tree_df[error_cols].abs()
                    .describe()
                    .loc[['mean', 'std']]
                    .T
                    .sort_values(by='mean'))

,mean,std
doubt_coverage_error,3.226723,3.311910
mapie_coverage_error,3.580289,2.489488
nasa_coverage_error,22.567957,9.561601
